In [ ]:
import pyrealsense2 as rs
import numpy as np
import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO
import torch

# ---------------------- 설정 ----------------------
BAG_FILE = r"/workspace/data/r2_o_l2.bag"   # 1번 코드의 경로 유지
MODEL_PATH = r"/workspace/mk/yolo11m-seg.pt"  # 1번 코드의 경로 유지

# 경보/표시 범위
STATIC_REPORT_RANGE  = 5.0    # 2번 코드 기본값 채택 (정적)
DYNAMIC_REPORT_RANGE = 7.0    # 2번 코드 기본값 채택 (동적)

# 동적/정적 판정 민감도
EPS_DEFAULT = 0.1
EPS_PERSON  = 0.05

# YOLO 추론 신뢰도
YOLO_CONF = 0.2

# ---------------------- 디바이스 설정 ----------------------
if torch.cuda.is_available():
    try:
        torch.cuda.set_device(0)  # GPU 0번 강제 (가능한 경우)
    except Exception as e:
        print(f"CUDA device set error: {e}")

# ---------------------- YOLO 모델 로드 ----------------------
model = YOLO(MODEL_PATH, task="segment")
if torch.cuda.is_available():
    model.to("cuda")

# ---------------------- RealSense 파이프라인 설정 ----------------------
pipeline = rs.pipeline()
config   = rs.config()

# 파일 입력 + 스트림 명시 (2번 코드 반영)
config.enable_device_from_file(BAG_FILE, repeat_playback=False)
config.enable_stream(rs.stream.depth, rs.format.z16, 30)
config.enable_stream(rs.stream.color, rs.format.rgb8, 30)
config.enable_stream(rs.stream.accel, rs.format.motion_xyz32f, 100)
config.enable_stream(rs.stream.gyro,  rs.format.motion_xyz32f, 200)

profile = pipeline.start(config)
align   = rs.align(rs.stream.color)
depth_scale = profile.get_device().first_depth_sensor().get_depth_scale()

# ---------------------- 전역 상태 ----------------------
prev_time        = None
velocity         = np.zeros(2, dtype=np.float32)
my_position      = np.zeros(2, dtype=np.float32)
last_my_position = np.zeros(2, dtype=np.float32)
prev_distances   = {}   # tid -> prev_distance
frame_count      = 0

# ---------------------- 함수 ----------------------
def _extract_accel(frames):
    """
    RealSense SDK 버전에 따라 first_or_default가 없을 수 있어 보강.
    가속도 프레임을 안전하게 찾아 반환.
    """
    # 새 API 시도
    try:
        ms = frames.first_or_default(rs.stream.accel)
        if ms:
            return ms.as_motion_frame().get_motion_data()
    except Exception:
        pass

    # 호환 경로: iterate
    try:
        for f in frames:
            if f.is_motion_frame() and f.get_profile().stream_type() == rs.stream.accel:
                return f.as_motion_frame().get_motion_data()
    except Exception:
        pass
    return None


def process_frame(frames):
    global prev_time, velocity, my_position, last_my_position, prev_distances, frame_count

    aligned = align.process(frames)
    dframe  = aligned.get_depth_frame()
    cframe  = aligned.get_color_frame()
    if not dframe or not cframe:
        return None, None

    # 영상/깊이
    rgb   = np.asanyarray(cframe.get_data())
    color = cv2.cvtColor(rgb, cv2.COLOR_RGB2BGR)
    depth = np.asanyarray(dframe.get_data()) * depth_scale
    h0, w0 = color.shape[:2]

    # 시간/IMU
    accel = _extract_accel(frames)

    t  = frames.get_timestamp() / 1000.0
    dt = 0.0 if prev_time is None else (t - prev_time)
    prev_time = t

    if accel is not None and dt > 0:
        v_acc       = np.array([accel.x, accel.y], dtype=np.float32)
        velocity[:] = np.clip(velocity + v_acc * dt, -1.0, 1.0)
        my_position += velocity * dt

    distance_travelled = float(np.linalg.norm(my_position - last_my_position))
    last_my_position[:] = my_position
    print(f"[Frame {frame_count}] I walked: {distance_travelled:.2f} m")

    # YOLO + ByteTrack
    try:
        results = model.track(color, tracker="bytetrack.yaml", persist=True, conf=YOLO_CONF)[0]
    except Exception as e:
        print(f"[Frame {frame_count}] YOLO error: {e}")
        frame_count += 1
        return color, []

    # 박스/ID 없는 프레임 처리
    if results.boxes is None or results.boxes.id is None or len(results.boxes) == 0:
        frame_count += 1
        return color, []

    boxes_tensor = results.boxes.data
    ids_tensor   = results.boxes.id
    boxes = boxes_tensor.detach().cpu().numpy()
    ids   = ids_tensor.detach().cpu().numpy().astype(int)

    # 세그멘테이션 마스크 (없을 수 있음)
    masks = None
    if getattr(results, "masks", None) is not None and results.masks is not None:
        if getattr(results.masks, "data", None) is not None:
            masks = results.masks.data.detach().cpu().numpy().astype(np.uint8)

    # 각 객체의 깊이/상태 계산
    distances = {}   # tid -> (cls_id, median_depth)
    objects_info = []

    # 먼저 깊이 추정
    for i, tid in enumerate(ids):
        cls_id = int(boxes[i][-1])

        # 깊이 샘플링: 마스크 있으면 마스크, 없으면 박스 중앙 ROI
        if masks is not None and i < len(masks):
            mask_full = cv2.resize(masks[i], (w0, h0), interpolation=cv2.INTER_NEAREST).astype(bool)
            vals = depth[mask_full]
        else:
            x1, y1, x2, y2 = boxes[i][:4].astype(int)
            x1 = max(0, min(w0-1, x1)); x2 = max(0, min(w0-1, x2))
            y1 = max(0, min(h0-1, y1)); y2 = max(0, min(h0-1, y2))
            if x2 <= x1 or y2 <= y1:
                continue
            # 박스 내부 중앙 50% ROI
            cx1 = x1 + (x2 - x1)//4
            cx2 = x2 - (x2 - x1)//4
            cy1 = y1 + (y2 - y1)//4
            cy2 = y2 - (y2 - y1)//4
            roi = depth[cy1:cy2, cx1:cx2]
            vals = roi.flatten()

        valid = vals[vals > 0]
        if valid.size == 0:
            continue

        median_d = float(np.median(valid))
        distances[tid] = (cls_id, median_d)

    # 상태 판정 + 시각화
    for tid, (cls_id, cur_d) in distances.items():
        prev_d = prev_distances.get(tid, cur_d)
        delta  = prev_d - cur_d  # +면 가까워짐, -면 멀어짐

        label_map = getattr(model, "names", None)
        label = label_map.get(cls_id, str(cls_id)) if isinstance(label_map, dict) else str(cls_id)

        is_person = (label == "person")
        eps = EPS_PERSON if is_person else EPS_DEFAULT
        thr = distance_travelled + eps

        # 동적/정적 + 이동 방향 태그
        if delta < -eps:
            state = "dynamic"
            motion_dir = " (away)"
        elif abs(delta) > thr:
            state = "dynamic"
            motion_dir = " (close)"
        else:
            state = "static"
            motion_dir = ""

        # 박스 좌표/중심
        try:
            idx = list(ids).index(tid)
        except ValueError:
            continue
        x1, y1, x2, y2 = boxes[idx][:4].astype(int)
        x1 = max(0, min(w0-1, x1)); x2 = max(0, min(w0-1, x2))
        y1 = max(0, min(h0-1, y1)); y2 = max(0, min(h0-1, y2))
        if x2 <= x1 or y2 <= y1:
            continue
        cx = int((x1 + x2) * 0.5)
        cy = int((y1 + y2) * 0.5)

        # 범위 색상: out-of-range(회색), 동적(빨강), 정적(파랑)
        in_range = (cur_d <= (DYNAMIC_REPORT_RANGE if state == "dynamic" else STATIC_REPORT_RANGE))
        if not in_range:
            color_box = (160, 160, 160)
            suffix = " (out-range)"
        else:
            color_box = (0, 0, 255) if state == "dynamic" else (255, 0, 0)
            suffix = ""

        # 시각화
        cv2.rectangle(color, (x1, y1), (x2, y2), color_box, 2)
        cv2.putText(
            color,
            f"{state.title()}{motion_dir} | {label} | {cur_d:.2f}m{suffix}",
            (x1, max(0, y1 - 10)),
            cv2.FONT_HERSHEY_SIMPLEX, 0.6, color_box, 2
        )

        # 로그 + 리턴용 정보 저장
        print(
            f"[Frame {frame_count}] Obj#{tid}({label}): "
            f"prev_d={prev_d:.2f}m, cur_d={cur_d:.2f}m, Δ={delta:.2f}m --> {state}{motion_dir}{suffix}"
        )
        objects_info.append({
            "tid": tid,
            "label": label,
            "distance": cur_d,
            "state": state,
            "cx": cx,
            "cy": cy,
            "in_range": in_range
        })

    # 이전 거리 업데이트
    prev_distances = {tid: dist for tid, (_, dist) in distances.items()}
    frame_count += 1

    return color, objects_info


def analyze_scene(color, objects_info):
    """
    1번 코드의 좌/중/우 혼잡도 기반 추천 방향 로직 유지.
    in_range인 객체만 고려해 추천 정확도를 높임.
    """
    if color is None or not objects_info:
        return

    h0, w0 = color.shape[:2]
    third = w0 // 3
    counts = {"left": 0, "center": 0, "right": 0}

    # 범위 내 객체만 카운트 (동적/정적 모두)
    for obj in objects_info:
        if not obj.get("in_range", True):
            continue
        if obj["cx"] < third:
            counts["left"] += 1
        elif obj["cx"] < 2 * third:
            counts["center"] += 1
        else:
            counts["right"] += 1

    best_dir = min(counts, key=counts.get)

    # 가까운 순으로 정렬(범위 내 우선)
    sorted_objs = sorted(
        objects_info,
        key=lambda x: (not x.get("in_range", True), x["distance"])
    )

    # 콘솔 출력
    print(f"추천 방향: {best_dir} (L={counts['left']}, C={counts['center']}, R={counts['right']})")
    print("==== 가까운 객체 순 (in-range 우선) ====")
    for obj in sorted_objs:
        tag = "" if obj.get("in_range", True) else " (out-range)"
        print(f"Obj#{obj['tid']} ({obj['label']}): {obj['distance']:.2f}m [{obj['state']}] {tag}")

    # 시각화 가이드라인
    lavender = (200, 150, 255)
    cv2.line(color, (third, 0), (third, h0), lavender, 2)
    cv2.line(color, (2 * third, 0), (2 * third, h0), lavender, 2)
    cv2.putText(color, f"Go {best_dir.upper()}", (50, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 0), 3)

    # Matplotlib 표시
    plt.figure(figsize=(8, 6))
    plt.imshow(cv2.cvtColor(color, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title(f"추천: {best_dir}")
    plt.show()
    plt.close('all')


# ---------------------- 실행 루프 ----------------------
try:
    while True:
        try:
            frames = pipeline.wait_for_frames(timeout_ms=5000)
        except RuntimeError:
            print("⚠️ .bag 파일 재생이 끝났습니다. 종료합니다.")
            break

        color, objects_info = process_frame(frames)
        analyze_scene(color, objects_info)

        # ESC 종료
        if cv2.waitKey(1) == 27:
            break
finally:
    pipeline.stop()
    cv2.destroyAllWindows()
